In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

Classify an image

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt

image_url = "https://upload.wikimedia.org/wikipedia/commons/a/aa/California_quail.jpg"
image_path = keras.utils.get_file(origin=image_url)
image = keras.utils.load_img(image_path)
plt.imshow(image)

In [ ]:
import keras_hub

image_classifier = keras_hub.models.ImageClassifier.from_preset(
    "resnet_50_imagenet",
    activation="softmax",
)

In [ ]:
batch = np.array([image])

In [ ]:
image_classifier.preprocessor.image_size = (224, 224)


In [ ]:
preds = image_classifier.predict(batch)
preds.shape

In [ ]:
keras_hub.utils.decode_imagenet_predictions(preds)

In [ ]:
print("Raw input shape:", batch.shape)
resized_batch = image_classifier.preprocessor(batch)
print("Preprocessed input shape:", resized_batch.shape)

In [ ]:
image_classifier.summary()

Generate text with an LLM

In [ ]:
causal_lm = keras_hub.models.CausalLM.from_preset(
    "gemma2_instruct_2b_en",
    dtype="bfloat16",
)

CausalLM come with an extra function called generate() which can be used generate predict tokens in a loop and decode them as a string.

In [ ]:
template = "<start_of_turn>user\n{question}<end_of_turn>\n<start_of_turn>model"

In [ ]:
question = """Write a python program to generate the first 1000 prime numbers.
Just show the actual code."""

In [ ]:
print(causal_lm.generate(template.format(question=question), max_length=512))

In [ ]:
question = "Share a very simple brownie recipe."
print(causal_lm.generate(template.format(question=question), max_length=512))

In [ ]:
causal_lm.summary()

In [ ]:
tokenizer = causal_lm.preprocessor.tokenizer
tokens_ids = tokenizer.tokenize("The quick brown fox jumps over the lazy dog.")
print(tokens_ids)

In [ ]:
string = tokenizer.detokenize(tokens_ids)
print(string)

In [ ]:
causal_lm.compile(
    sampler=keras_hub.samplers.TopKSampler(k=10, temperature=2.0),
)

question = "Share a very simple brownie recipe."
print(causal_lm.generate(template.format(question=question), max_length=512))

Fine-tune and publish an image classifier